# Grocery Favorita: AWS DeepAR vs Custom Seq2Seq

<br>
<br>
<br>

<img src="https://upload.wikimedia.org/wikipedia/commons/0/0f/Corporaci%C3%B3n_Favorita_Logo.png"> 

<br>
<br>

## Overview
This notebook covers creating features (tensors and embeddings) for the Seq2Seq model. Only the top 5% most popular store-item pairs are considered for predictions; popularity is defined by the number of unique days sales took place. Additionally, the only features created are those that are readily available in the data provided; this is to benchmark the performance of a custom model versus the performance the deepAR timeseries model that AWS provides. Features are either instance specific (sales, promotion occurring, item details, store details) or shared among instances across dates (datetime details, oil price)   

## Imports

In [291]:
import pandas as pd
import numpy as np
import os
import datetime as dt
import dask.dataframe as ddf
from embedder.preprocessing import (categorize, pick_emb_dim, encode_categorical)
from embedder.regression import Embedder
from embedder.assessment import visualize
import feather
import plotly.offline

## Feature Engineering
### Functions

In [2]:
# creates datetime featues when datetime column is given
def generate_datetimedf(df_datetime_col):
    
    from workalendar.usa import California
    from dateutil import easter
    import calendar
    import datetime as dt

    first_date = min(df_datetime_col)
    last_date = max(df_datetime_col)
    all_years = (df_datetime_col.apply(lambda x: x.year)).unique()

    cal = California()
    all_holidays = []
    for i in all_years:
        holidays = [x[0] for x in cal.holidays(i)]
        holidays = holidays+[easter.easter(i)]
        all_holidays.append(holidays)

    all_holidays = set([y for x in all_holidays for y in x])

    weekends = set(['Saturday', 'Sunday'])
    datetimedf = pd.DataFrame({
#         'datetime_calendar_year': df_datetime_col.apply(lambda x: x.year),
#                                'datetime_calendar_month': df_datetime_col.apply(lambda x: x.month),
#                                'datetime_calendar_day': df_datetime_col.apply(lambda x: x.day),
#                                'datetime_calendar_hour': df_datetime_col.apply(lambda x: x.hour),
#                                'datetime_day_of_year': df_datetime_col.apply(lambda x: x.timetuple().tm_yday),
                               'datetime_day_of_week': df_datetime_col.apply(lambda x: x.timetuple().tm_wday),
#                                'datetime_day_of_cycle': df_datetime_col.apply(lambda x: (x-first_date).days+1),
                               'datetime_is_weekend': df_datetime_col.apply(lambda x: 1 if calendar.day_name[x.weekday()] in weekends else 0),
#                                'datetime_is_holiday': df_datetime_col.apply(lambda x: 1 if x.date() in all_holidays else 0)
                               })

#     holiday_1d = [x+dt.timedelta(days=1) for x in list(all_holidays)] + \
#         [x+dt.timedelta(days=-1) for x in list(all_holidays)]
#     holiday_2d = holiday_1d+[x+dt.timedelta(days=2) for x in list(all_holidays)]+[
#         x+dt.timedelta(days=-2) for x in list(all_holidays)]
#     holiday_3d = holiday_2d+[x+dt.timedelta(days=3) for x in list(all_holidays)]+[
#         x+dt.timedelta(days=-3) for x in list(all_holidays)]
#     datetimedf['datetime_1d_away_holiday'] = df_datetime_col.apply(
#         lambda x: 1 if x.date() in set(holiday_1d) else 0)
#     datetimedf['datetime_2d_away_holiday'] = df_datetime_col.apply(
#         lambda x: 1 if x.date() in set(holiday_2d) else 0)
#     datetimedf['datetime_3d_away_holiday'] = df_datetime_col.apply(
#         lambda x: 1 if x.date() in set(holiday_3d) else 0)
#     holiday_7d = holiday_3d

#     for d in [4, 5, 6, 7]:
#         for i in [x+dt.timedelta(days=d) for x in list(all_holidays)]:
#             holiday_7d.append(i)
#         for t in [x+dt.timedelta(days=-d) for x in list(all_holidays)]:
#             holiday_7d.append(t)

#     datetimedf['datetime_within_7d_holiday'] = df_datetime_col.apply(
#         lambda x: 1 if x.date() in set(holiday_7d) else 0)
    
    datetimedf.index = df_datetime_col

    return datetimedf

In [3]:
def read_csv_generator(path,preview_csv=False):
    csvs = [x for x in os.listdir(path) if 'csv' in x]
    
    if not preview_csv:
        for csv in csvs:
            print(
                "%s_df = pd.read_csv('%s/%s')" % (csv.replace('.csv',
                                                              ''), path, csv)
            )
    else:
        for csv in csvs:
            print(csv)
            display(pd.read_csv('%s/%s' % (path, csv) ).head() )

### Loading Data

In [4]:
# # first time loading data
# types_dict = {'id': 'int32',
#              'item_nbr': 'int32',
#              'store_nbr': 'int8',
#              'unit_sales': 'float32'}

# train_df = pd.read_csv('data/train.csv', low_memory=True, dtype=types_dict, parse_dates=['date'])

In [5]:
# second time loading data
train_df = feather.read_dataframe('data/train_feather')

In [6]:
read_csv_generator('data')

transactions_df = pd.read_csv('data/transactions.csv')
items_df = pd.read_csv('data/items.csv')
oil_df = pd.read_csv('data/oil.csv')
holidays_events_df = pd.read_csv('data/holidays_events.csv')
stores_df = pd.read_csv('data/stores.csv')


In [7]:
transactions_df = pd.read_csv('data/transactions.csv')
items_df = pd.read_csv('data/items.csv')
oil_df = pd.read_csv('data/oil.csv')
holidays_events_df = pd.read_csv('data/holidays_events.csv')
stores_df = pd.read_csv('data/stores.csv')

In [8]:
read_csv_generator('data',preview_csv=1)

transactions.csv


,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


items.csv


,item_nbr,family,class,perishable
0,96995,GROCERY I,1093,0
1,99197,GROCERY I,1067,0
2,103501,CLEANING,3008,0
3,103520,GROCERY I,1028,0
4,103665,BREAD/BAKERY,2712,1


oil.csv


,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


holidays_events.csv


,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


stores.csv


,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


### Filter to the Most Popular Store / Item Pairs

In [10]:
most_popular_store_items = train_df.groupby(
    ['store_nbr', 'item_nbr'])['date'].nunique()

In [11]:
most_popular_store_items.sort_values(ascending=False).describe()

count    174685.000000
mean        718.419097
std         514.831287
min           1.000000
25%         245.000000
50%         651.000000
75%        1133.000000
max        1679.000000
Name: date, dtype: float64

In [12]:
most_popular_store_items_df = most_popular_store_items[most_popular_store_items > np.percentile(most_popular_store_items, 95)].sort_values(ascending=False).reset_index()

In [13]:
%time

store_nbrs = set(list(most_popular_store_items_df['store_nbr']))
item_nbrs = set(list(most_popular_store_items_df['item_nbr']))

train_df = train_df[train_df.store_nbr.isin(store_nbrs)]
train_df = train_df[train_df.item_nbr.isin(item_nbrs)]

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 9.06 µs


### Instance Specific Features
- Sales
- Promotion
- Item Details (Tile By Number of Dates)
- Store Details (Tile By Number of Dates)

In [14]:
%%time
most_popular_store_items_df['raw_data'] = most_popular_store_items_df.apply(lambda row: train_df[(train_df.store_nbr == row['store_nbr']) & (train_df.item_nbr == row['item_nbr'])], axis=1)

CPU times: user 11min 46s, sys: 4min 25s, total: 16min 11s
Wall time: 16min 13s


In [15]:
# only create features between the latest earliest date and the earliest latest date for all instances
recent_min_date = most_popular_store_items_df['raw_data'].apply(lambda x: x['date'].min()).max()
latest_max_date = most_popular_store_items_df['raw_data'].apply(lambda x: x['date'].max()).min()

print(recent_min_date, latest_max_date)

(Timestamp('2013-03-16 00:00:00'), Timestamp('2017-07-04 00:00:00'))


In [16]:
date_range_df=pd.DataFrame({'date':  pd.date_range(recent_min_date,latest_max_date,freq='D') })

#### Embeddings
- at the store and item detail level
- not done at the datetime level unless needed

In [17]:
# embedding lookups

emb_store_lookup = stores_df[stores_df.store_nbr.isin(store_nbrs)].set_index('store_nbr').astype(str)
emb_item_lookup = items_df[items_df.item_nbr.isin(item_nbrs)].set_index('item_nbr').astype(str)


In [18]:
# must create embeddings using the latest date of each category: of these choose the earliest date
min_max_emb_date = min(list(train_df.groupby(['store_nbr'])['date'].max().values) + list(train_df.groupby(['item_nbr'])['date'].max().values))

In [20]:
%time
def generate_embeddings():
    # take only data points for the 2w for the training period for embeddings for timesake
    X = train_df[(train_df['date'] >= (latest_max_date + dt.timedelta(-7)) + dt.timedelta(-56)) & (train_df['date'] <= (latest_max_date + dt.timedelta(-7)) )].merge(
        generate_datetimedf(date_range_df['date']).applymap(str).reset_index(), on=['date'], how='left'
    ).merge(
        stores_df, on=['store_nbr'], how='left'
    ).merge(
        items_df, on=['item_nbr'], how= 'left'
    )

    emb_target = X['unit_sales'].apply(lambda x: 0 if x < 0 else x).apply(lambda x: np.log1p(x))

    X = X.drop(columns=['id','date','unit_sales']).applymap(str)

    cat_vars = categorize(X)
    embedding_dict = pick_emb_dim(cat_vars, max_dim=50)
    X_encoded, encoders = encode_categorical(X)

    embedder = Embedder(embedding_dict)

    embedder.fit(X_encoded, emb_target)
    embeddings = embedder.get_embeddings()

    return embeddings, encoders

embeddings, encoders = generate_embeddings()

CPU times: user 4 µs, sys: 3 µs, total: 7 µs
Wall time: 11.2 µs
Train on 1215573 samples, validate on 303894 samples
Epoch 1/100
1215573/1215573 [==============================] - 58s 47us/step - loss: 0.3011 - r2: 0.5929 - val_loss: 0.2702 - val_r2: 0.5631
Epoch 2/100
1215573/1215573 [==============================] - 59s 48us/step - loss: 0.2526 - r2: 0.6584 - val_loss: 0.2774 - val_r2: 0.5534
Epoch 3/100
1215573/1215573 [==============================] - 57s 47us/step - loss: 0.2413 - r2: 0.6737 - val_loss: 0.2551 - val_r2: 0.5873
Epoch 4/100
1215573/1215573 [==============================] - 60s 49us/step - loss: 0.2351 - r2: 0.6820 - val_loss: 0.2498 - val_r2: 0.5957
Epoch 5/100
1215573/1215573 [==============================] - 61s 50us/step - loss: 0.2314 - r2: 0.6871 - val_loss: 0.2474 - val_r2: 0.5999
Epoch 6/100
1215573/1215573 [==============================] - 62s 51us/step - loss: 0.2282 - r2: 0.6913 - val_loss: 0.2476 - val_r2: 0.5987
Epoch 7/100
1215573/1215573 [========

Epoch 58/100
1215573/1215573 [==============================] - 88s 73us/step - loss: 0.1940 - r2: 0.7374 - val_loss: 0.2585 - val_r2: 0.5804
Epoch 59/100
1215573/1215573 [==============================] - 89s 73us/step - loss: 0.1938 - r2: 0.7378 - val_loss: 0.2586 - val_r2: 0.5805
Epoch 60/100
1215573/1215573 [==============================] - 88s 72us/step - loss: 0.1936 - r2: 0.7381 - val_loss: 0.2594 - val_r2: 0.5792
Epoch 61/100
1215573/1215573 [==============================] - 98s 80us/step - loss: 0.1933 - r2: 0.7382 - val_loss: 0.2581 - val_r2: 0.5816
Epoch 62/100
1215573/1215573 [==============================] - 105s 86us/step - loss: 0.1933 - r2: 0.7385 - val_loss: 0.2557 - val_r2: 0.5850
Epoch 63/100
1215573/1215573 [==============================] - 102s 84us/step - loss: 0.1931 - r2: 0.7387 - val_loss: 0.2593 - val_r2: 0.5799
Epoch 64/100
1215573/1215573 [==============================] - 102s 84us/step - loss: 0.1929 - r2: 0.7390 - val_loss: 0.2596 - val_r2: 0.5787
Epo

In [21]:
# extract embedding data
def gernerate_all_embedding_lookup_df():
    emb_key_df = pd.DataFrame(embeddings.keys(), columns=['key'])

    def generate_embedding_df(col_):
    #     if embeddings[col_].shape < 3:
        col_emb_df = pd.DataFrame(
            embeddings[col_],
            index=encoders[col_].classes_,
        ).rename_axis(col_).reset_index()
    #     else:
    
        col_emb_df.columns = [col_] + [col_ + '_' + str(x) for x in list(col_emb_df) if x != col_]

        return col_emb_df

    emb_key_df['emb_data'] = emb_key_df['key'].apply(lambda x: generate_embedding_df(x))

    return emb_key_df

all_embedding_lookup_df = gernerate_all_embedding_lookup_df()

In [22]:
# merge embedding columns to .a master lookup
def generate_master_instance_lookup():

    master_instance_lookup = most_popular_store_items_df[['store_nbr','item_nbr']].merge(stores_df).merge(items_df).astype(str)

    for df in all_embedding_lookup_df['emb_data']:
        
        try:
            master_instance_lookup = master_instance_lookup.merge(df.astype(str))
    
        except:
            pass
            print list(df)
            
            
    master_instance_lookup['store_nbr'] = master_instance_lookup['store_nbr'].astype(int)
    master_instance_lookup['item_nbr'] = master_instance_lookup['item_nbr'].astype(int)

    return master_instance_lookup

master_instance_lookup = generate_master_instance_lookup()


[u'onpromotion', u'onpromotion_0']
['datetime_day_of_week', 'datetime_day_of_week_0', 'datetime_day_of_week_1', 'datetime_day_of_week_2', 'datetime_day_of_week_3']
['datetime_is_weekend', 'datetime_is_weekend_0']


In [195]:
def create_instance_specific_features(sales_df):

    selected_item = sales_df['item_nbr'].values[0]
    selected_store = sales_df['store_nbr'].values[0]

    sales_df['unit_sales'] = sales_df['unit_sales'].apply(
        lambda x: 0 if x < 0 else x)

    # date range returns sales
    sales_date_range_df = date_range_df.merge(sales_df[['date', 'unit_sales']], how='left').set_index(
        'date').T.reset_index().rename(columns={'index': 'store_item'})
    sales_date_range_df['store_item'] = 'store_%s_item_%s' % (
        sales_df['store_nbr'].values[0], sales_df['item_nbr'].values[0])

    # promotion tensor

    promo_df = date_range_df.merge(
        sales_df[['date', 'onpromotion']], how='left')
    promo_df['onpromotion'] = promo_df['onpromotion'].fillna(
        False).apply(lambda x: 1 if x else 0)

    promo_tensor = promo_df['onpromotion'].values.reshape(
        -1, 1).reshape(1, len(promo_df), 1)
    
    store_item_details_tensors = np.tile(
    np.expand_dims(

        master_instance_lookup.loc[
            (master_instance_lookup['item_nbr'] == selected_item) & (
                master_instance_lookup['store_nbr'] == selected_store),
            [col for col in [col for col in list(master_instance_lookup) if '_' in col] if (
                col != 'store_nbr') & (col != 'item_nbr')]
        ].astype(float), axis=0

    ),     (1, date_range_df.shape[0], 1))
    
    return sales_date_range_df, promo_tensor, store_item_details_tensors

In [212]:
%%time
features = most_popular_store_items_df.apply(lambda row: create_instance_specific_features(row['raw_data']), axis=1)

# %%time
# df_dask_most_popular_store_items_df = ddf.from_pandas(most_popular_store_items_df, npartitions=10) 
# features = df_dask_most_popular_store_items_df.apply(lambda row: create_instance_specific_features(row['raw_data']), axis=1, meta=('object') ).compute(scheduler='multiprocessing')

all_sales_df = pd.concat([i[0] for i in features]).reset_index(drop=True)
promo_tensors = np.concatenate([i[1] for i in features], axis=0)
store_item_tensors = np.concatenate([i[2] for i in features], axis=0)

CPU times: user 4min 49s, sys: 45.8 s, total: 5min 35s
Wall time: 6min 14s


### Shared Datetime Tensors
- Oil Price
- Datetime Details

In [59]:
# oil price

oil_df['date']=pd.to_datetime(oil_df['date'])

oil_df = date_range_df.merge(oil_df,how='left')

oil_df['dcoilwtico']=oil_df['dcoilwtico'].interpolate()

oil_df['dcoilwtico']=oil_df['dcoilwtico'].interpolate().fillna(method='bfill')

oil_df['date'].describe()

oil_values = (oil_df['dcoilwtico'].apply(lambda x: np.log1p(
            x)) - oil_df['dcoilwtico'].apply(lambda x: np.log1p(x)).mean()).values

oil_tensors = oil_values.reshape(-1, 1).reshape(1, len(oil_values), 1)

oil_tensors = np.tile(oil_tensors, (most_popular_store_items_df.shape[0],1,1) )

In [61]:
# datetime tensors

datetime_dummy_lookup = pd.get_dummies(generate_datetimedf(date_range_df['date']).applymap(str), drop_first=True).reset_index()
# datetime_dummy_lookup.iloc[:,1:]
dt_tensors = np.expand_dims(datetime_dummy_lookup.iloc[:,1:].values, axis=0)

dt_tensors = np.tile(dt_tensors, (most_popular_store_items_df.shape[0],1,1) )

In [ ]:
np.savez_compressed('exports/store_item_tensors',store_item_tensors)

In [33]:
np.savez_compressed('exports/promo_tensors',promo_tensors)

In [38]:
np.savez_compressed('exports/oil_tensors',oil_tensors)

In [39]:
np.savez_compressed('exports/dt_tensors',dt_tensors)

In [30]:
all_sales_df.to_pickle('exports/all_sales_df.pkl')

## Analysis of Results

In [294]:
pd.DataFrame({
    'model': ['aws_deepAR_just_sales','seq_just_sales','seq_full'],
    'mae': [8.627952, 6.153596, 6.656257]
})[['model','mae']]

,model,mae
0,aws_deepAR_just_sales,8.627952
1,seq_just_sales,6.153596
2,seq_full,6.656257


The custom seq2seq model performs 28.67% better than AWS's deepAR algorithm in MAE. Other out-of-the-box features did not help the model's performance, meaning that improvements to the model could only be made by creating more complex features, such as lags across groupings, or parameter tuning. 